In [1]:
!ls /content/drive/MyDrive/NLP\ assignment\ 3/

cc.en.300.vec  ExtractedWords.json			 __pycache__		       utils.py
ClfDatasets    fastTextSelfExtro_threshold95_dim300.txt  Text_Classification_mv.ipynb
dataloader.py  glove.42B.300d.txt			 TextCNN.py


In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import re
import operator

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import sys
sys.path.append('/content/drive/MyDrive/NLP assignment 3/')
import utils

from tqdm import tqdm_notebook
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
!unzip -o /content/drive/MyDrive/NLP\ assignment\ 3/ClfDatasets/IMDB.zip -d /content/drive/MyDrive/NLP\ assignment\ 3/ClfDatasets/IMDB/


Archive:  /content/drive/MyDrive/NLP assignment 3/ClfDatasets/IMDB.zip
warning [/content/drive/MyDrive/NLP assignment 3/ClfDatasets/IMDB.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
file #1:  bad zipfile offset (local header sig):  4
file #2:  bad zipfile offset (local header sig):  39
file #3:  bad zipfile offset (local header sig):  93
file #4:  bad zipfile offset (lseek):  13541376
 extracting: /content/drive/MyDrive/NLP assignment 3/ClfDatasets/IMDB/IMDB/unlabeledTrainData.tsv  


In [4]:
!ls /content/drive/MyDrive/NLP\ assignment\ 3/ClfDatasets/IMDB/IMDB

unlabeledTrainData.tsv


In [5]:
!unzip /content/drive/MyDrive/NLP\ assignment\ 3/ClfDatasets/AGNews.zip -d /content/drive/MyDrive/NLP\ assignment\ 3/ClfDatasets/AGNews/


Archive:  /content/drive/MyDrive/NLP assignment 3/ClfDatasets/AGNews.zip
replace /content/drive/MyDrive/NLP assignment 3/ClfDatasets/AGNews/AGNews/classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/MyDrive/NLP assignment 3/ClfDatasets/AGNews/AGNews/classes.txt  
  inflating: /content/drive/MyDrive/NLP assignment 3/ClfDatasets/AGNews/AGNews/readme.txt  
  inflating: /content/drive/MyDrive/NLP assignment 3/ClfDatasets/AGNews/AGNews/test.csv  
  inflating: /content/drive/MyDrive/NLP assignment 3/ClfDatasets/AGNews/AGNews/train.csv  


In [6]:
!ls /content/drive/MyDrive/NLP\ assignment\ 3/ClfDatasets/AGNews/AGNews

classes.txt  readme.txt  test.csv  train.csv


In [7]:
# DBpedia, YahooAnswersUpper, YahooAnswersLower, YelpReviews, IMDB, AGNews
x_train, y_train, x_test, y_test, TopicList, Idx2Topic = utils.LoadDatasets("AGNews")
Idx2Topic_list = []
for i in range(len(set(TopicList))): Idx2Topic_list.append(Idx2Topic[i])
Idx2Topic_list = np.array(Idx2Topic_list)

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [8]:
WordDict = {"<NONE>":0, "<OOV>":1}
WordCnt = {}

data_train, WordDict, WordCnt, MaxSeqLen = utils.DataProcessing(x_train, WordDict, WordCnt, TrainFlag=True)
data_valid, _, _, _ = utils.DataProcessing(x_valid, WordDict, WordCnt, TrainFlag=False)
data_test, _, _, _ = utils.DataProcessing(x_test, WordDict, WordCnt, TrainFlag=False)

WordCnt = sorted(WordCnt.items(), key=operator.itemgetter(1))
WordIdx = dict(zip(WordDict.values(), WordDict.keys()))
x_train_emb = utils.EmbeddingNumpy(data_train, WordDict, MaxSeqLen)
x_valid_emb = utils.EmbeddingNumpy(data_valid, WordDict, MaxSeqLen)
x_test_emb = utils.EmbeddingNumpy(data_test, WordDict, MaxSeqLen)
y_train = np.asarray(y_train)
y_valid = np.asarray(y_valid)
y_test = np.asarray(y_test)

NumClass = max(len(set(y_train)), max(y_train)+1)

Processing Data:   0%|          | 0/102000 [00:00<?, ?it/s]

Processing Data:   0%|          | 0/18000 [00:00<?, ?it/s]

Processing Data:   0%|          | 0/7600 [00:00<?, ?it/s]

Creating Embeddings:   0%|          | 0/102000 [00:00<?, ?it/s]

Creating Embeddings:   0%|          | 0/18000 [00:00<?, ?it/s]

Creating Embeddings:   0%|          | 0/7600 [00:00<?, ?it/s]

In [9]:
import dataloader
import copy
import random

### Hyperparameters
Hyperparams = {
    "NumClfEpoch" : 100,
    "EmbeddingSize" : 300, # 300, 768
    "KernelSize" : [2,3,4,5],
    "BatchSize" : 2**8,
    "LearningRate" : 1e-3,
    "ChannelSize" : [32, 16],
#     "UsePreWordVector" : False,
#     "UsePreWordVector" : "../../Data/PretrainedWV/glove.42B.300d.txt",
#     "UsePreWordVector" : "../../Data/PretrainedWV/GloVeFullExtro1.txt",
    "UsePreWordVector": "/content/drive/MyDrive/NLP assignment 3/fastTextSelfExtro_threshold95_dim300.txt",
    "WordVectorNorm" : False,
    "MaxSeqLen" : MaxSeqLen,
    "NumClass" : NumClass,
}

WordIdx = dict(zip(WordDict.values(), WordDict.keys()))
train_dataset = dataloader.ClassifyDataset(x=x_train_emb, y=y_train)
valid_dataset = dataloader.ClassifyDataset(x=x_valid_emb, y=y_valid)
test_dataset = dataloader.ClassifyDataset(x=x_test_emb, y=y_test)
train_loader = DataLoader(dataset=train_dataset, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)

In [11]:
import os
os.makedirs("./save", exist_ok=True)

In [ ]:
import TextCNN
from imp import reload
reload(TextCNN)

for it in range(5):
    print(it+1)

    model = TextCNN.ConvNet(WordDict, Hyperparams); model = model.to(device)
    crit_crossentropy = nn.CrossEntropyLoss()
    optimizer_cnn = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=Hyperparams["LearningRate"])

    # Train the model
    maxacc_all = 0; Step = 0
    maxacc_val = 0; EarlyStopCnt = 5
    total_step = len(train_loader)
    pbar1 = tqdm_notebook(total = Hyperparams["NumClfEpoch"], leave=False, desc="Epoch")
    model.train() # train mode
    for epoch in range(Hyperparams["NumClfEpoch"]):
        pbar1.update(1)
        pbar2 = tqdm_notebook(total = total_step*Hyperparams["BatchSize"], leave=False, desc="Training")
        for i, (texts, labels) in enumerate(train_loader):
            pbar2.update(Hyperparams["BatchSize"])
            texts = texts.to(device); labels = labels.to(device)
            outputs = model(texts)
            loss_cnn = crit_crossentropy(outputs, labels)
            optimizer_cnn.zero_grad()
            loss_cnn.backward()
            optimizer_cnn.step()
        pbar2.close()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, Hyperparams["NumClfEpoch"], loss_cnn.item()), end=' ')

        # Valid the model
        model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
        with torch.no_grad():
            correct = 0.; total = 0
            for texts, labels in valid_loader:
                texts = texts.to(device); labels = labels.to(device)
                outputs = model(texts)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            if (correct/total) >= maxacc_val:
                maxacc_val = correct/total
                torch.save(model.state_dict(), "./save/model")
                torch.save(model.embedding.weight, "./save/emb")

            else: # (correct/total) < maxacc_val:
                EarlyStopCnt = EarlyStopCnt-1
                if EarlyStopCnt == 0: break
            print('ValidAcc: {:.4f} % , MaxAcc: {:.4f}'.format(100 * correct/total, maxacc_val))

    ### Test Acc.
    model = TextCNN.ConvNet(WordDict, Hyperparams).to(device)
    model.load_state_dict(torch.load("./save/model"))
    model.embedding.weight.data = torch.load("./save/emb")

    with torch.no_grad():
        model.eval()
        correct = 0.; total = 0
        for texts, labels in test_loader:
            texts = texts.to(device); labels = labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('TestAcc: {:.4f} % , ValidAcc: {:.4f}'.format(100 * correct / total, maxacc_val))
    pbar1.close()
    #     print("MaxAcc: {:.4f}".format(maxacc))

1
Loading Pretrained Word Vectors ... 
53703 Word Vectors Loaded . 


<ipython-input-12-311f76352cca>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar1 = tqdm_notebook(total = Hyperparams["NumClfEpoch"], leave=False, desc="Epoch")


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-12-311f76352cca>:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar2 = tqdm_notebook(total = total_step*Hyperparams["BatchSize"], leave=False, desc="Training")


Training:   0%|          | 0/102144 [00:00<?, ?it/s]

...

Epoch [8/100], Loss: 0.0561 Loading Pretrained Word Vectors ... 
53703 Word Vectors Loaded . 
TestAcc: 90.7105 % , ValidAcc: 0.9117
5
Loading Pretrained Word Vectors ... 
53703 Word Vectors Loaded . 


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/102144 [00:00<?, ?it/s]

Epoch [1/100], Loss: 0.3694 ValidAcc: 89.8333 % , MaxAcc: 0.8983


Training:   0%|          | 0/102144 [00:00<?, ?it/s]